In [1]:
from ECCV18_initializers import *
import Networks as STNet
import SearchTools as search
from ReportGenerator import R_Recall_at_T, mAP_at_k
from scipy.spatial.distance import cdist
import scipy.sparse as sps
import time

In [2]:
###########################################################
# Train the model:
model_pretrain = STNet.BaseLearner(k,L,m=m,Learner=Learner,nlinStrategy='KBest_STC')
model_pretrain.run(F_train)
print(' ---------- distortion vs. number of bits for training set:')
print(model_pretrain.distortion)
print(model_pretrain.rate*n)

 ################## Starting to learn network parameters:  ##################
layer-units:
** 1 **** 2 **** 3 **** 4 **** 5 **** 6 **** 7 **** 8 **** 9 **
Finished learning network parameters:
 ---------- distortion vs. number of bits for training set:
[ 1.          0.19276921  0.07762106  0.03607568  0.02900506  0.02366239
  0.01937174  0.01585071  0.01298979  0.01064467]
[   0.           69.29709831  155.45016866  248.78085882  275.03846935
  301.51224918  328.10274028  354.66898781  381.25542906  407.82330912]


In [3]:
####################################################
# Encoding the entire database based on the trained parameters:
obj_F = STNet.fwdPass(model_pretrain.params,k,nlinStrategy='KBest_STC')
_,F_hat,X = obj_F.run(F)
for l in range(L):
    X[l] = sps.csc_matrix((X[l]))
print(' ---------- distortion vs. number of bits for the main database:')
print(obj_F.distortion)
print(obj_F.rate*n)

 Running the network:
 ##################  Running the network:  ##################
layer-units:
** 1 **** 2 **** 3 **** 4 **** 5 **** 6 **** 7 **** 8 **** 9 **
Finished running the network.
 ---------- distortion vs. number of bits for the main database:
[ 1.          0.19261927  0.07763419  0.03608795  0.02904271  0.02370401
  0.01940824  0.01588608  0.01302185  0.01067161]
[   0.           69.20689281  155.35767536  248.75004114  275.03884578
  301.53732217  328.14167101  354.72349462  381.32521628  407.9101207 ]


In [4]:
############################################################
# Encoding the queries based on the trained parameters:
obj_Q = STNet.fwdPass(model_pretrain.params,k,nlinStrategy='KBest_STC')
_,_,Y = obj_Q.run(Q)
for l in range(L):
    Y[l] = sps.csc_matrix((Y[l]))

print(obj_Q.distortion)
print(obj_Q.rate*n)

 Running the network:
 ##################  Running the network:  ##################
layer-units:
** 1 **** 2 **** 3 **** 4 **** 5 **** 6 **** 7 **** 8 **** 9 **
Finished running the network.
[ 1.          0.19354975  0.0780981   0.03631615  0.02921068  0.02380542
  0.01947944  0.01595261  0.01305724  0.01069532]
[   0.           69.23310043  155.5943655   248.87906763  275.10526432
  301.52307293  328.05062597  354.56814881  381.07445966  407.59448813]


In [5]:
##############################3
# ANN databases have 3 sets: train, main and query. Other databases
# like MNIST have train and test only.
if trainset_mainset_seperated:
    Distortion = obj_F.distortion
    if rate_calc_exct:
        BitList = [0]
        for l in range(L):
            alphabet_probs = \
                STNet.combinatorial_binomial_poisson_distributions\
                    (1 - obj_F.prob_z[l], obj_F.nlinParam[l])
            BitList.append(STNet.categorical_entropy(alphabet_probs) + obj_F.nlinParam[l])
        BitList = np.cumsum(BitList)
    else:
        BitList = obj_F.rate * n
else:
    Distortion = obj_Q.distortion
    if rate_calc_exct:
        BitList = [0]
        for l in range(L):
            alphabet_probs = \
                STNet.combinatorial_binomial_poisson_distributions\
                    (1 - obj_Q.prob_z[l], obj_Q.nlinParam[l])
            BitList.append(STNet.categorical_entropy(alphabet_probs) + obj_Q.nlinParam[l])
        BitList = np.cumsum(BitList)
    else:
        BitList = obj_Q.rate * n
print('BitList = \n',BitList)

BitList = 
 [   0.           69.20689281  155.35767536  248.75004114  275.03884578
  301.53732217  328.14167101  354.72349462  381.32521628  407.9101207 ]


In [6]:
##################################
# Initial decoding based on multi-layer codes:
Recat1_initial = []
Recat10_initial = []
Recat100_initial = []
mAP10_initial = []
mAP100_initial = []
Recat1_refined = []
Recat10_refined = []
Recat100_refined = []
mAP10_refined = []
mAP100_refined = []
#
Votes = np.zeros((N_f, N_q))
# Heuristically weight the codes from different layers:
# Vote_weight = obj_Q.distortion[0:-1] / obj_Q.distortion[1:]
Vote_weight = obj_Q.distortion[1:] ** 2
Vote_weight[vote_layer_dismiss:] = 0
for l in range(L):
    print(' ***************** Decoding layer l =',l+1, ' ****************')
    ############################################################
    # Initial decoding from STC:
    if Vote_weight[l] != 0:
        t = time.time()
        tempVotes = Vote_weight[l] * search.ternaryVoter(X[l], Y[l], nu, nu_prime).toarray()
        # tempVotes = STNet.nlinearity(tempVotes, 10000, 'KBest')
        Votes += tempVotes
        List_initial = np.argsort(-Votes, axis=0)[:initial_list_size, :]
        print('elapsed time for initial decoding = ', time.time() - t)
    #
    Recat1_initial.append(R_Recall_at_T(Y_NNs, List_initial, R=1, T=1))
    Recat10_initial.append(R_Recall_at_T(Y_NNs, List_initial, R=10, T=10))
    Recat100_initial.append(R_Recall_at_T(Y_NNs, List_initial, R=100, T=100))
    mAP10_initial.append(mAP_at_k(Y_NNs[0:10,:], List_initial[0:10,:], k=10))
    mAP100_initial.append(mAP_at_k(Y_NNs[0:100,:], List_initial[0:100,:], k=100))
    print('initial 1-Recall@1 = ', Recat1_initial[-1])
    print('initial mAP@100 = ', mAP100_initial[-1])
    ############################################################
    t = time.time()
    # Final decoding based on reconstructions:
    List_refined = search.List_refiner(np.sum(F_hat[0:l+1], axis=0), Q, List_initial)
    #
    Recat1_refined.append(R_Recall_at_T(Y_NNs, List_refined, R=1, T=1))
    Recat10_refined.append(R_Recall_at_T(Y_NNs, List_refined, R=10, T=10))
    Recat100_refined.append(R_Recall_at_T(Y_NNs, List_refined, R=100, T=100))
    mAP10_refined.append(mAP_at_k(Y_NNs[0:10,:], List_refined[0:10,:], k=10))
    mAP100_refined.append(mAP_at_k(Y_NNs[0:100,:], List_refined[0:100,:], k=100))
    print('elapsed time for list refinement = ', time.time() - t)
    print('refined 1-Recall@1 = ', Recat1_refined[-1])
    print('refined mAP@100 = ', mAP100_refined[-1])

print(' ***************** Finished the decoding ****************')

 ***************** Decoding layer l = 1  ****************
elapsed time for initial decoding =  224.68936491012573
initial 1-Recall@1 =  0.12
initial mAP@100 =  0.0917953685330356
elapsed time for list refinement =  13.353838920593262
refined 1-Recall@1 =  0.21
refined mAP@100 =  0.20462926246659863
 ***************** Decoding layer l = 2  ****************
elapsed time for initial decoding =  318.0556619167328
initial 1-Recall@1 =  0.17
initial mAP@100 =  0.12694416878735115
elapsed time for list refinement =  14.037405014038086
refined 1-Recall@1 =  0.467
refined mAP@100 =  0.5037658990043645
 ***************** Decoding layer l = 3  ****************
elapsed time for initial decoding =  260.25286746025085
initial 1-Recall@1 =  0.179
initial mAP@100 =  0.13198275759667039
elapsed time for list refinement =  14.593857288360596
refined 1-Recall@1 =  0.624
refined mAP@100 =  0.6700595769573531
 ***************** Decoding layer l = 4  ****************
elapsed time for initial decoding =  131

In [ ]:
##################################################################
# Saving the results:
suffix = '_k' + str(int(np.mean(k))) + '_L' + str(L) + '_m' + str(int(np.mean(m))) + '_'+ Learner
fname = '/' + Database + '_D_Bit'  + suffix
np.savetxt(PGF_path + fname + '.dat', np.transpose([BitList, Distortion]), fmt='%8f')
#
fname = '/' + Database + '_Recat1_initial_Bit' + suffix
np.savetxt(PGF_path + fname + '.dat', np.transpose([BitList[1:], Recat1_initial]), fmt='%8f')
#s
fname = '/' + Database + '_Recat10_initial_Bit' + suffix
np.savetxt(PGF_path + fname + '.dat', np.transpose([BitList[1:], Recat10_initial]), fmt='%8f')
#
fname = '/' + Database + '_Recat100_initial_Bit' + suffix
np.savetxt(PGF_path + fname + '.dat', np.transpose([BitList[1:], Recat100_initial]), fmt='%8f')
#
fname = '/' + Database + '_mAP10_initial_Bit' + suffix
np.savetxt(PGF_path + fname + '.dat', np.transpose([BitList[1:], mAP10_initial]), fmt='%8f')
#
fname = '/' + Database + '_mAP100_initial_Bit' + suffix
np.savetxt(PGF_path + fname + '.dat', np.transpose([BitList[1:], mAP100_initial]), fmt='%8f')
#

fname = '/' + Database + '_Recat1_refined_Bit' + suffix
np.savetxt(PGF_path + fname + '.dat', np.transpose([BitList[1:], Recat1_refined]), fmt='%8f')
#
fname = '/' + Database + '_Recat10_refined_Bit' + suffix
np.savetxt(PGF_path + fname + '.dat', np.transpose([BitList[1:], Recat10_refined]), fmt='%8f')
#
fname = '/' + Database + '_Recat100_refined_Bit' + suffix
np.savetxt(PGF_path + fname + '.dat', np.transpose([BitList[1:], Recat100_refined]), fmt='%8f')
#
fname = '/' + Database + '_mAP10_refined_Bit' + suffix
np.savetxt(PGF_path + fname + '.dat', np.transpose([BitList[1:], mAP10_refined]), fmt='%8f')
#
fname = '/' + Database + '_mAP100_refined_Bit' + suffix
np.savetxt(PGF_path + fname + '.dat', np.transpose([BitList[1:], mAP100_refined]), fmt='%8f')


In [ ]:
# If you want to free RAM after the experiments are finished:
import os
os._exit(00)